<a href="https://colab.research.google.com/github/ACEHUNTER128/test/blob/main/AdvancedProgramming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Part 01: TensorFlow (with the Task 2 included in)

In [42]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [43]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

In [44]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [45]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, batch_size=32)

#Applying Task 2 in the Tensor Flow Part
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy (TF): {test_acc:.4f}")

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8573 - loss: 0.4962
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9555 - loss: 0.1584
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9667 - loss: 0.1139
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9750 - loss: 0.0843
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9796 - loss: 0.0688
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9663 - loss: 0.1127
Test Accuracy (TF): 0.9720


In [47]:
#Task 3: TensorFlow conversion to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpmfdxad2s'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor_8')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  136090200470352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136090200460560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136090200470160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136090196817104: TensorSpec(shape=(), dtype=tf.resource, name=None)


Part 2: PyTorch

In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [62]:
# Load and preprocess data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Scale to [-1, 1]
])
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

In [63]:
# Define model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 64)
        self.fc2 = nn.Linear(64, 10)
    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten
        x = torch.relu(self.fc1(x))
        return self.fc2(x)  # Logits (no softmax for CrossEntropyLoss)

model = Net()
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [64]:
# Training loop
for epoch in range(5):
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 100 == 99:
            print(f"Epoch {epoch+1}, Batch {batch_idx+1}: Loss = {running_loss / 100:.4f}")
            running_loss = 0.0

# Save the trained model for Task 2
torch.save(model.state_dict(), 'mnist_model.pth')

Epoch 1, Batch 100: Loss = 1.1978
Epoch 1, Batch 200: Loss = 0.5206
Epoch 1, Batch 300: Loss = 0.4783
Epoch 1, Batch 400: Loss = 0.4010
Epoch 1, Batch 500: Loss = 0.3770
Epoch 1, Batch 600: Loss = 0.3797
Epoch 1, Batch 700: Loss = 0.3491
Epoch 1, Batch 800: Loss = 0.3235
Epoch 1, Batch 900: Loss = 0.3446
Epoch 1, Batch 1000: Loss = 0.3153
Epoch 1, Batch 1100: Loss = 0.3400
Epoch 1, Batch 1200: Loss = 0.3186
Epoch 1, Batch 1300: Loss = 0.3376
Epoch 1, Batch 1400: Loss = 0.2960
Epoch 1, Batch 1500: Loss = 0.2873
Epoch 1, Batch 1600: Loss = 0.2934
Epoch 1, Batch 1700: Loss = 0.2930
Epoch 1, Batch 1800: Loss = 0.2656
Epoch 2, Batch 100: Loss = 0.2533
Epoch 2, Batch 200: Loss = 0.2555
Epoch 2, Batch 300: Loss = 0.2441
Epoch 2, Batch 400: Loss = 0.2487
Epoch 2, Batch 500: Loss = 0.2336
Epoch 2, Batch 600: Loss = 0.2588
Epoch 2, Batch 700: Loss = 0.2304
Epoch 2, Batch 800: Loss = 0.2240
Epoch 2, Batch 900: Loss = 0.2438
Epoch 2, Batch 1000: Loss = 0.2246
Epoch 2, Batch 1100: Loss = 0.2248
Epo

Now for Task 2 in PyTorch

In [65]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms

In [66]:
# Load test data (same preprocessing as Task 1)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
test_data = datasets.MNIST(root='./data', train=False, transform=transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1000)

In [67]:
# Rebuild the model architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 64)
        self.fc2 = nn.Linear(64, 10)
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)


In [68]:
# Load the saved model from Task 1
model = Net()
model.load_state_dict(torch.load('mnist_model.pth'))
model.eval()  # Set to evaluation mode

Net(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

In [69]:
# Evaluation
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()

print(f"Test Accuracy: {correct / len(test_loader.dataset):.4f}")

Test Accuracy: 0.9556


In [77]:
%pip install onnx

In [78]:
#Task 3 Conversion to ONNX
import onnx
dummy_input = torch.randn(1, 1, 28, 28)  # Batch of 1 image
torch.onnx.export(model, dummy_input, "model.onnx",
                  input_names=["input"], output_names=["output"])